### Обновление библиотек

In [ ]:
!pip install phik
!pip install shap
!pip install lightgbm
!pip install -U scikit-learn

### Импорт библиотек

In [ ]:


import warnings
from warnings import simplefilter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap
from lightgbm import LGBMRegressor, LGBMClassifier
from phik import phik_matrix
from phik.report import plot_correlation_matrix
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler 
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVR  # Вместо SVC используем SVR

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
simplefilter(action='ignore', category=FutureWarning)


from IPython.display import display, HTML #Используется для прорисовки координат SHAP
import plotly.express as px
from plotly.subplots import make_subplots  # Исправленный импорт
from sklearn.feature_selection import SelectKBest, f_classif 


### Константы

In [ ]:
# Константы
RANDOM_STATE = 42
line_1 = '-'*125 #линия 
line_2 = '_'*125#линия

### Переменные

In [ ]:
#Задача №1
#Выделем дискретные и непрерывные количественныз переменные
discrete_columns =['employment_years', 'supervisor_evaluation']
continuous_columns = ['job_satisfaction_rate', 'salary']

#Категориальные переменные 
categorical_features = ['dept','level','workload','last_year_promo','last_year_violations']

#Задача №2                        
#Выделем дискретные и непрерывные количественныз переменные
quit_discrete_columns = ['employment_years','supervisor_evaluation','quit']
quit_continuous_columns = ['salary']
#Категориальные переменные 
quit_categorical_features = ['dept','level','workload','workload','last_year_violations','last_year_promo']

### Функции

#### Функция об общей информации о датасете

In [ ]:
def get_info(data):
    """
    Функция выводит основную информацию о датасете: первые 5 строк, общую информацию, основные статистики,
    количество явных дубликатов и количество пропусков

    Args:
        data(pandas.DataFrame): Датасет 

    Returns:
        None

    """
    display(data.head())
    display(data.info())
    display(data.describe().T)
    print('Кол-во явных дубликатов:', data.duplicated().sum())
    print('Кол-во пропусков:\n', data.isna().sum())

#### Функция для анализа количественных переменных в разбивке на дискретные и непрерывные

In [ ]:
## Функция для анализа количественных переменных в разбивке на дискретные и непрерывные

def plot_histograms_for_df(df, name, discrete_columns, continuous_columns):
    """
    Функция для построения графиков для всех указанных дискретных и непрерывных столбцов в датафрейме.
    Дискретные признаки визуализируются с помощью countplot, а непрерывные - с помощью histplot с линиями среднего и медианы.
    
    Параметры:
    - df: DataFrame с данными.
    - name: Строка, которая будет добавлена к заголовку каждого подграфика.
    - discrete_columns: Список названий дискретных признаков.
    - continuous_columns: Список названий непрерывных признаков.
    """
    sns.set(style="whitegrid")

    # Определяем все столбцы для визуализации
    columns = discrete_columns + continuous_columns

    # Определяем количество строк и столбцов для подграфиков
    n_cols = 2  # Количество столбцов подграфиков
    n_rows = (len(columns) + n_cols - 1) // n_cols  # Количество строк подграфиков

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten()  # Преобразуем в 1D массив для удобной итерации

    for ax, column in zip(axes, columns):
        if column in discrete_columns:
            # Дискретные признаки
            sns.countplot(x=column, data=df, ax=ax)
            ax.set_title(f'{name} - {column} (дискретный)', fontsize=12)
            ax.set_xlabel(column, fontsize=10)
            ax.set_ylabel('Частота', fontsize=10)
        elif column in continuous_columns:
            # Непрерывные признаки
            sns.histplot(data=df, x=column, bins=20, kde=True, ax=ax)
            ax.set_title(f'{name} - {column} (непрерывный)', fontsize=12)
            ax.set_xlabel(column, fontsize=10)
            ax.set_ylabel('Частота', fontsize=10)

            # Добавляем медиану и среднее значение
            median_value = df[column].median()
            mean_value = df[column].mean()
            ax.axvline(median_value, color='r', linestyle='--', label=f'Медиана: {median_value:.2f}')
            ax.axvline(mean_value, color='g', linestyle='-', label=f'Среднее: {mean_value:.2f}')
            ax.legend(fontsize=8)

    # Удаляем лишние подграфики, если они есть
    total_plots = n_rows * n_cols
    if len(columns) < total_plots:
        for ax in axes[len(columns):]:
            fig.delaxes(ax)

    plt.tight_layout()
    plt.show()

""" 
# Пример вызова функции:
discrete_columns = ['колонка1', 'колонка2']
continuous_columns = ['колонка3', 'колонка4']
plot_histograms_for_df(df, 'Данные', discrete_columns, continuous_columns)

"""

#### Функция визулизации выбросов для непрерывных и дискретных данных

In [ ]:
#Функция визулизации выбросов для непрерывных и дискретных данных

def plot_interactive_plots(df, name, continuous_columns, discrete_columns):
    """
    Функция для построения интерактивных графиков:
    - boxplot для непрерывных признаков
    - countplot для дискретных признаков

    Параметры:
    - df: DataFrame с данными.
    - name: Строка, которая будет добавлена к заголовку каждого подграфика.
    - continuous_columns: Список названий непрерывных признаков. Если None, используются все числовые признаки из DataFrame.
    - discrete_columns: Список названий дискретных признаков. Если None, используются все дискретные (нечисловые) признаки из DataFrame.
    
    Описание:
    Функция строит интерактивные boxplot для непрерывных признаков и countplot для дискретных признаков.
    Если каких-либо признаков нет, то графики для них не строятся.
    """

    # Удаляем столбец 'id', если он присутствует в данных
    df_copy = df.drop(columns='id', errors='ignore')

    # Если не указаны списки признаков, по умолчанию все числовые признаки считаются непрерывными, а остальные — дискретными
    if continuous_columns is None:
        continuous_columns = df_copy.select_dtypes(include=['int64', 'float64']).columns.tolist()
    if discrete_columns is None:
        discrete_columns = df_copy.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

    # Определяем количество графиков для непрерывных и дискретных признаков
    n_continuous = len(continuous_columns)
    n_discrete = len(discrete_columns)
    
    total_plots = n_continuous + n_discrete
    
    # Если нет признаков для построения, выводим сообщение
    if total_plots == 0:
        print("Нет признаков для построения графиков")
        return

    # Создаем сетку подграфиков, исходя из общего числа графиков
    fig = make_subplots(rows=(total_plots + 1) // 2, cols=2, subplot_titles=[f"{name} - {col}" for col in continuous_columns + discrete_columns])

    # Построение boxplot для непрерывных признаков
    row, col = 1, 1
    for column in continuous_columns:
        boxplot = px.box(df_copy, y=column).data[0]
        fig.add_trace(boxplot, row=row, col=col)
        
        # Переход на следующую колонку/строку
        if col == 1:
            col = 2
        else:
            col = 1
            row += 1

    # Построение countplot для дискретных признаков
    for column in discrete_columns:
        countplot = px.histogram(df_copy, x=column).data[0]
        fig.add_trace(countplot, row=row, col=col)
        
        # Переход на следующую колонку/строку
        if col == 1:
            col = 2
        else:
            col = 1
            row += 1

    # Настройка размера и заголовка
    fig.update_layout(
        height=300 * ((total_plots + 1) // 2),  # Высота графиков динамически изменяется в зависимости от количества графиков
        width=1000,  # Ширина полотна
        title_text=f'Interactive Plots of Features for {name}'  # Общий заголовок графиков
    )

    # Отображение графиков
    fig.show()

"""
# Пример вызова функции
discrete_columns = ['колонка1', 'колонка2']
continuous_columns = ['колонка3', 'колонка4']
plot_interactive_plots(df, 'Данные', continuous_columns, discrete_columns)

"""


#### Функция визулизации категориальных данных

In [ ]:
def plot_categorical_features(df, name, categorical_features):
    """
    Функция для построения круговых диаграмм для категориальных признаков.

    Параметры:
    - df: DataFrame с данными.
    - categorical_features: Список категориальных признаков, которые необходимо визуализировать.
    - name: Строка, которая будет добавлена к заголовку каждого подграфика.

    Описание:
    Строит круговые диаграммы для категориальных признаков, используя сетку графиков.
    """
    # Подсчитываем общее количество категориальных признаков
    total_plots = len(categorical_features)

    # Задаем количество строк и столбцов для подграфиков, исходя из количества категориальных признаков
    rows = (total_plots + 1) // 2
    fig, axes = plt.subplots(rows, 2, figsize=(15, 5 * rows))  # Изменяем высоту сетки, чтобы учесть количество графиков
    axes = axes.ravel()  # Преобразуем 2D массив осей в 1D, чтобы легче было итерироваться

    plot_index = 0  # Индекс для выбора текущей оси

    # Итерация по каждому категориальному признаку
    for column in categorical_features:
        if column in df.columns:  # Проверяем, что колонка существует в DataFrame
            df[column].value_counts().plot.pie(
                ax=axes[plot_index], autopct='%1.1f%%', startangle=90, fontsize=12
            )
            axes[plot_index].set_title(f'{name} - {column}', fontsize=16)  # Добавляем name к заголовку
            axes[plot_index].set_ylabel('')  # Убираем метку оси Y
            plot_index += 1

    # Удаляем лишние оси, если они есть
    if plot_index < len(axes):
        for ax in axes[plot_index:]:
            fig.delaxes(ax)

    # Настраиваем внешний вид диаграмм
    plt.tight_layout()
    plt.show()

"""
# Пример использования:
categorical_features = ['Category1', 'Category2', 'Category3']
plot_categorical_features(df,'Данные',categorical_features)
"""


#### Функция для корреляционного анализа

In [ ]:
def perform_correlation_analysis(df, discrete_columns, continuous_columns):
    """
    Функция для выполнения корреляционного анализа с использованием библиотеки phik
    и построения тепловой карты для всех данных.

    Аргументы:
    - df: DataFrame с данными.
    - discrete_columns: Список дискретных признаков.
    - continuous_columns: Список непрерывных признаков.

    Пример использования:
    --------------------
    # Пример списков с дискретными и непрерывными признаками
    discrete_columns = ['dept', 'level', 'workload']
    continuous_columns = ['salary', 'age', 'working_hours']

    # Вызов функции для выполнения корреляционного анализа
    perform_correlation_analysis(df, discrete_columns, continuous_columns)
    """

    # Удаляем столбец 'id', так как он не нужен для анализа
    df_clean = df.drop('id', axis=1, errors='ignore')

    # Построение общей корреляционной матрицы для всех данных
    print("\nОбщая корреляционная матрица:")
    corr_matrix = df_clean.phik_matrix(interval_cols=continuous_columns).round(2)

    # Отображение общей тепловой карты
    plot_correlation_matrix(
        corr_matrix.values,
        x_labels=corr_matrix.columns,
        y_labels=corr_matrix.index,
        vmin=0, vmax=1, color_map='Greens',
        title=r'Корреляция $\phi_K$',
        fontsize_factor=1.5,
        figsize=(20, 15)
    )

def plot_correlation_matrix(corr_matrix, x_labels, y_labels, vmin, vmax, color_map, title, fontsize_factor, figsize):
    """
    Вспомогательная функция для построения тепловой карты корреляций.

    Аргументы:
    - corr_matrix: Матрица корреляций.
    - x_labels: Метки по оси X.
    - y_labels: Метки по оси Y.
    - vmin, vmax: Минимальное и максимальное значение корреляции.
    - color_map: Цветовая схема.
    - title: Заголовок графика.
    - fontsize_factor: Фактор увеличения шрифтов.
    - figsize: Размер графика.
    """
    plt.figure(figsize=figsize)
    sns.heatmap(corr_matrix, annot=True, cmap=color_map, vmin=vmin, vmax=vmax, xticklabels=x_labels, yticklabels=y_labels)
    plt.title(title, fontsize=fontsize_factor * 10)
    plt.xticks(rotation=45, ha='right', fontsize=fontsize_factor * 8)
    plt.yticks(fontsize=fontsize_factor * 8)
    plt.tight_layout()
    plt.show()

    """
    Пример использования:
    --------------------
    # Пример списков с дискретными и непрерывными признаками
    discrete_columns = ['dept', 'level', 'workload']
    continuous_columns = ['salary', 'age', 'working_hours']

    # Вызов функции для выполнения корреляционного анализа
    perform_correlation_analysis(df, discrete_columns, continuous_columns) 
    """

#### Функция для построения графиков зависимости увольнения от признаков в датасете.

In [ ]:
def plot_quit_analysis_in_columns(data, target='quit', plot_type='box', cols=2):
    """
    Функция для автоматического построения графиков зависимости между категориальными и числовыми признаками
    с учётом целевого признака (например, увольнения) и выводом графиков в две колонки.
    
    Параметры:
    ----------
    data: DataFrame
        Датасет для анализа (например, quit_X_train).
    
    target: str, optional
        Целевой признак для анализа (по умолчанию 'quit').

    plot_type: str, optional
        Тип графика ('box', 'violin'). Зависит от типа данных.

    cols: int, optional
        Количество колонок для отображения графиков (по умолчанию 2).
    
    Возвращает:
    -----------
    None
        Построит и покажет графики для всех категориальных признаков с числовыми.
    """
    # Получаем список всех категориальных и числовых признаков
    categorical_columns = data.select_dtypes(include=['object', 'category']).columns.tolist()
    numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Убираем целевой признак из списков
    if target in categorical_columns:
        categorical_columns.remove(target)
    if target in numerical_columns:
        numerical_columns.remove(target)

    # Инициализация для многоколоночного вывода
    total_plots = len(categorical_columns) * len(numerical_columns)
    rows = (total_plots + cols - 1) // cols  # Определение количества строк для всех графиков
    
    # Создаем подграфики
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()  # Преобразуем в плоский массив для удобства работы с осями

    plot_index = 0  # Счетчик для отслеживания номера графика

    # Проходим по каждому категориальному и числовому признаку и строим графики
    for cat_col in categorical_columns:
        for num_col in numerical_columns:
            plt.sca(axes[plot_index])  # Переключаемся на конкретный подграфик
            
            if plot_type == 'box':
                sns.boxplot(x=cat_col, y=num_col, hue=target, data=data, ax=axes[plot_index])
            elif plot_type == 'violin':
                sns.violinplot(x=cat_col, y=num_col, hue=target, data=data, split=True, ax=axes[plot_index])
            else:
                raise ValueError("Неверный тип графика. Доступны: 'box', 'violin'")
            
            # Устанавливаем заголовок для графика
            axes[plot_index].set_title(f'{num_col} по {cat_col} с учётом {target}')
            plot_index += 1

    # Удаляем пустые подграфики, если общее количество графиков меньше, чем количество подграфиков
    for i in range(plot_index, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()


# Задача 1: предсказание уровня удовлетворённости сотрудника

## Шаг 1.1. Загрузка данных

### 1.1.1. Загрузка данных

In [ ]:
# Загружаем датасеты для первой задачи
try:
    job_X_train = pd.read_csv('datasets/train_job_satisfaction_rate.csv')
    job_X_test = pd.read_csv('datasets/test_features.csv')
    job_y_test = pd.read_csv('datasets/test_target_job_satisfaction_rate.csv')

except:
    job_X_train = pd.read_csv('https://code.s3.yandex.net/datasets/train_job_satisfaction_rate.csv')
    job_X_test = pd.read_csv('https://code.s3.yandex.net/datasets/test_features.csv')
    job_y_test = pd.read_csv('https://code.s3.yandex.net/datasets/test_target_job_satisfaction_rate.csv')


### 1.1.2. Общая информация о датасетах

#### `job_X_train`

In [ ]:
get_info(job_X_train)

#### `job_X_test`

In [ ]:
get_info(job_X_test)

#### `job_y_test`

In [ ]:
get_info(job_y_test)

### Общий вывод по датасетам **job_X_train**, **job_X_test** и **job_y_test**:

1. **job_X_train** (Тренировочная выборка для предсказания уровня удовлетворённости):
- **Размер**: 4000 записей и 10 столбцов.
- **Признаки**:
  - Категориальные: **dept**, **level**, **workload**, **last_year_promo**, **last_year_violations**.
  - Числовые: **id**, **employment_years**, **supervisor_evaluation**, **salary**, **job_satisfaction_rate** (целевой признак).
- **Пропуски**: Есть пропуски в признаках **dept** (отдел) и **level** (уровень должности).
- **Целевой признак**: **job_satisfaction_rate** — уровень удовлетворённости сотрудника.

2. **job_X_test** (Тестовая выборка с признаками):
- **Размер**: 2000 записей и 9 столбцов (без целевого признака).
- **Признаки**:
  - Категориальные: **dept**, **level**, **workload**, **last_year_promo**, **last_year_violations**.
  - Числовые: **id**, **employment_years**, **supervisor_evaluation**, **salary**.
- **Пропуски**: Есть пропуски в колонках **dept** и **level**.
- **Целевой признак отсутствует**, так как это тестовая выборка для модели, которая будет предсказывать **job_satisfaction_rate**.

3. **job_y_test** (Целевой признак тестовой выборки):
- **Размер**: 2000 записей и 2 столбца (**id** и **job_satisfaction_rate**).
- **Пропуски отсутствуют**: Все данные целевого признака присутствуют и могут быть использованы для оценки модели.

**Общий вывод:**
- **Пропуски**: Пропуски имеются в категориальных признаках **dept** и **level** как в тренировочной, так и в тестовой выборке. Их нужно будет обработать на этапе предобработки данных.
- **Целевой признак**: Целевой признак **job_satisfaction_rate** присутствует в тренировочной выборке и тестовом наборе меток (**job_y_test**), что позволяет использовать его для обучения моделей и оценки их точности.
- **Разнородность данных**: Датасеты содержат как числовые, так и категориальные признаки, поэтому потребуется кодирование категориальных признаков перед обучением модели.
- **Целостность данных**: За исключением небольших пропусков в категориальных признаках, данные в хорошем состоянии для дальнейшей работы, включая анализ и построение моделей.


## Шаг 1.2. Предобработка данных

In [ ]:
# Пайп замены пропущенных значений на часто частое используемое (strategy='most_frequent')
# imputer_pipe = Pipeline(
    # [
        # (
#             'SimpleImputer',
#             SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#         ),
#     ]
# )

### `job_X_train`

In [ ]:
# Выявление неявных дубликатов. Создадим словарь с уникальными значениями для каждого категориального признака
unique_values_dict = {col: job_X_train[col].unique().tolist() for col in job_X_train.columns if job_X_train[col].dtype == 'object'}
unique_values_dict

**Вывод:**
1. ошибка в слове 'sinior'
2. Пропуски (NaN) присутствуют в колонках dept и level   

In [ ]:
# Повторное исправление опечатки в колонке level
job_X_train['level'] = job_X_train['level'].replace('sinior', 'senior')

# Проверим уникальные значения в колонке 'level' после исправления
job_X_train['level'].unique()


In [ ]:
# Применим пайплайн к этим двум столбцам
# job_X_train[['dept', 'level']] = imputer_pipe.fit_transform(job_X_train[['dept', 'level']])

# Проверим, что пропуски были обработаны
# job_X_train[['dept', 'level']].isnull().sum()

### `job_X_test`

In [ ]:
# Выявление неявных дубликатов. Создадим словарь с уникальными значениями для каждого категориального признака
unique_values_dict = {col: job_X_test[col].unique().tolist() for col in job_X_test.columns if job_X_test[col].dtype == 'object'}
unique_values_dict

Видими пустые ячейки в 'dept','workload' в виде '' и 'level' - nan

In [ ]:
#Исправляем ошибку
job_X_test['level'].replace({'sinior': 'senior'}, inplace=True)

In [ ]:
# Заменим "пустое" значение `dept` и `workload` на NaN
job_X_test['dept'].replace({' ': np.nan}, inplace=True)
job_X_test['workload'].replace({' ': np.nan}, inplace=True)

In [ ]:
# Обработаем пропущенные значения при помощи пайплайна 
# job_X_test[['dept','level']] = imputer_pipe.transform(job_X_test[['dept','level']])


In [ ]:
#Отдельно проработаем workload
# job_X_train[['workload']] = imputer_pipe.fit_transform(job_X_train[['workload']])
# job_X_test[['workload']] = imputer_pipe.transform(job_X_test[['workload']])

In [ ]:

# Проверка
display(job_X_test[job_X_test['dept'].isnull() | job_X_test['level'].isnull() | job_X_test['workload'].isnull()])
print(f'Количество пропущенных значений после замены: \n {job_X_test.isna().sum()}')

#### `job_y_test`

In [ ]:
job_y_test['job_satisfaction_rate'].unique()


Выводы по предобраьотке:

**`job_X_train`**
- Исправили*sinior -> senior* в столбце `level`
- Заменили пропуски при помощи пайплайна (`SimpleImputer`)

**`job_X_test`**
- Исправили *sinior -> senior* в столбце `level`
- Заменили пустые значения (" ")
- Заменили пропуски ари помощи пайплайна (`SimpleImputer`)


**`job_y_test`**
- Без изменений


## Шаг 1.3. Исследовательский анализ данных

#### Количественные данные

In [ ]:
#Гистограммы
plot_histograms_for_df(job_X_train,'Данные', discrete_columns, continuous_columns)

#Выбросы
plot_interactive_plots(job_X_train,'Данные', discrete_columns, continuous_columns)


#### Категориальные данные

In [ ]:
plot_categorical_features(job_X_train,'Данные', categorical_features)

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Я ревьюер простой: вижу графики признаков - пишу зелёный комментарий.

#### Корреляционный анализ

In [ ]:
perform_correlation_analysis(job_X_train, discrete_columns, continuous_columns)

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Красивый ковёр задаёт стиль всей работе! (с) Джеффри Лебовски

Говоря по-ревьюерски, молодец, что не забыл про корреляционный анализ :)

### Выводы EDA

1. **Распределение признаков**

*Дискретные признаки:*
- **Отдел (dept)**: Больше всего сотрудников работает в отделе продаж (38%), за ним следует отдел технологий (21.6%) и закупок (15.3%). Наименьшее количество сотрудников в HR (11.4%) и маркетинге (13.8%).
- **Уровень должности (level)**: Основная часть сотрудников имеет уровень junior (47.5%), затем идут сотрудники среднего уровня (middle) — 43.6%, и наименьшая часть — старшие сотрудники (senior), составляющие 9%.
- **Загруженность (workload)**: Большинство сотрудников имеют среднюю загруженность (51.6%), при этом загруженность на низком уровне у 30% сотрудников, и только 18.4% сотрудников имеют высокую загруженность.
- **Повышение за последний год (last_year_promo)**: Лишь 3% сотрудников получили повышение за последний год, что указывает на редкость данного события.
- **Нарушения за последний год (last_year_violations)**: 86% сотрудников не нарушали трудовой договор, и только 14% имели нарушения.
- **Оценка руководителя (supervisor_evaluation)**: Большинство сотрудников (около 45%) получили оценку "4" от руководителя, затем идут сотрудники с оценкой "3". Оценки "5" и "1" встречаются реже.

*Непрерывные признаки:*
- **Длительность работы (employment_years)**: Наиболее распространённый срок работы сотрудников — 1–2 года. Меньшее количество сотрудников имеет длительный срок работы более 6 лет.
- **Зарплата (salary)**: Средняя зарплата составляет около 33,927 условных единиц. Большая часть сотрудников зарабатывает в диапазоне 20,000–40,000. Видны признаки положительной асимметрии — зарплаты выше 60,000 встречаются реже.
- **Уровень удовлетворённости (job_satisfaction_rate)**: Удовлетворённость сотрудников варьируется от низкого до высокого уровня. Средняя удовлетворённость составляет около 0.53, медиана — 0.56, что говорит о том, что распределение близко к симметричному, с небольшой положительной асимметрией.

2. **Корреляционный анализ**
На тепловой карте корреляций видно следующие ключевые моменты:
- **Значимая корреляция между уровнем должности (level) и зарплатой (salary)**: Коэффициент корреляции составляет 0.72, что указывает на сильную положительную связь — чем выше уровень должности, тем выше зарплата.
- **Корреляция между уровнем должности и длительностью работы (employment_years)**: Коэффициент корреляции составляет 0.68, что логично — сотрудники с более высоким уровнем должности, как правило, дольше работают в компании.
- **Корреляция между уровнем должности и загруженностью (workload)**: Коэффициент 0.42 указывает на умеренную корреляцию — с повышением должности, увеличивается и уровень загруженности.
- **Оценка руководителя и уровень удовлетворённости**: Видна высокая положительная корреляция (0.76) между оценкой руководителя и уровнем удовлетворённости сотрудника, что указывает на важную роль положительной обратной связи для мотивации сотрудников.

**Заключение:**

- **Основные факторы, влияющие на уровень удовлетворённости**: Среди факторов, оказывающих влияние на уровень удовлетворённости сотрудников, особенно выделяются оценка руководителя, зарплата и длительность работы в компании.
- **Уровень должности играет важную роль**: Зависимость между уровнем должности и удовлетворённостью работы тесно связана с другими факторами, такими как зарплата, загруженность и длительность работы.
- **Редкость повышения и нарушения**: Лишь небольшое количество сотрудников нарушают трудовой договор и получают повышение за последний год, что может быть связано с внутренней политикой компании.

**Рекомендации:**
- Провести дополнительные исследования факторов, влияющих на удовлетворённость, возможно, с применением машинного обучения для предсказания уровня удовлетворённости.
- Проанализировать влияние редкости повышения и возможные пути увеличения мотивации через развитие карьерных возможностей.


## Шаг 1.4. Подготовка данных

### Предподгатовка

In [ ]:
# Подготовку признаков выполните в пайплайне, дополнив пайплайн шага предобработки.
# При кодировании учитывайте особенности признаков и моделей
#  и используйте как минимум два кодировщика.

In [ ]:
#Проверим размерность
# Оптимизированная проверка размерностей DataFrame'ов
for name, dataframe in zip(['job_X_train', 'job_X_test', 'job_y_test'],
                           [job_X_train, job_X_test, job_y_test]):
    print(f"Размер {name}: {dataframe.shape}")

In [ ]:
# Тренеровочные данные
## Создали скопировали датасет job_X_train и опрелели индекс
job_X_train_ml = job_X_train.copy().set_index('id')

## Выдели целевой признак job_satisfaction_rate из job_X_train_ml
job_y_train_ml = job_X_train_ml['job_satisfaction_rate']

##Удалим целевой принак из тренеровояной базы
job_X_train_ml = job_X_train_ml.drop('job_satisfaction_rate', axis=1)

In [ ]:
#Проверка размерности
job_X_train_ml.shape[0]-job_y_train_ml.shape[0]


In [ ]:
#Тестовые данные
job_X_test_ml = job_X_test.set_index('id')
job_y_test_ml = job_y_test.set_index('id')
job_full_test = job_X_test_ml.merge(job_y_test_ml, on='id', how='left')

## Выдели целевой признак job_satisfaction_rate
job_y_test_ml = job_full_test['job_satisfaction_rate']

##Удалим целевой принак из тестовой базы
job_X_test_ml = job_full_test.drop('job_satisfaction_rate', axis=1)


In [ ]:
#Проверим размерность
# Оптимизированная проверка размерностей DataFrame'ов
for name, dataframe in zip(['job_X_train_ml', 'job_y_train_ml', 'job_X_test_ml','job_y_test_ml'],
                           [job_X_train_ml, job_y_train_ml, job_X_test_ml,job_y_test_ml]):
    print(f"Размер {name}: {dataframe.shape}")

In [ ]:
display('Количество дубликатов job_X_train_ml после корректировок:', job_X_train_ml.duplicated().sum())


In [ ]:
#Убирем дубликаты из job_X_train_ml
job_X_train_ml.drop_duplicates(inplace=True, ignore_index=False)

#Синхронизируем job_y_train_ml с job_X_train_ml по индексу после удаления дубликато
job_y_train_ml = job_y_train_ml[job_y_train_ml.index.isin(job_X_train_ml.index)]


display('Количество дубликатов job_X_train_ml после корректировок:', job_X_train_ml.duplicated().sum())

In [ ]:
#Проверим размерность
# Оптимизированная проверка размерностей DataFrame'ов
for name, dataframe in zip(['job_X_train_ml', 'job_y_train_ml', 'job_X_test_ml','job_y_test_ml'],
                           [job_X_train_ml, job_y_train_ml, job_X_test_ml,job_y_test_ml]):
    print(f"Размер {name}: {dataframe.shape}")

## Шаг 1.5. Обучение моделей

#### Строим пайплан

In [ ]:
# 1. Создаём списки с названиями признаков
ohe_columns = ['dept', 'last_year_promo', 'last_year_violations']
ord_columns = ['level', 'workload']
num_columns = ['employment_years', 'supervisor_evaluation', 'salary']

In [ ]:
# 2. Заполнение пропусков SimpleImputer и OHE-кодирование
ohe_pipe = Pipeline(
    [
        # Шаг 1: Заполнение пропусков наиболее часто встречающимся значением
        (
            'simpleImputer_ohe',  # Название шага
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')  # Заполнение пропусков
        ),
        # Шаг 2: OneHotEncoder для кодирования категориальных признаков
        (
            'ohe',  # Название шага
            OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')  # One-hot кодирование
        )
    ]
)

ohe_pipe

In [ ]:
# 3. Заполнение пропусков SimpleImputer и Ordinal-кодирование (OE)
ord_pipe = Pipeline(
    [
        # Шаг 1: Замена пропущенных значений на наиболее часто встречающееся значение
        (
            'simpleImputer_before_ord',  # Название шага
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        # Шаг 2: Кодирование категориальных признаков порядковыми значениями
        (
            'ord',  # Название шага
            OrdinalEncoder(
                categories=[['junior', 'middle', 'senior'], ['low', 'medium', 'high']],  # Категории для кодирования
                #handle_unknown='use_encoded_value', unknown_value=-1  # Используем -1 для неизвестных категорий
                handle_unknown='use_encoded_value', unknown_value=np.nan
            )
        ),
        # Шаг 3: Замена пропущенных значений на наиболее часто встречающееся значение после кодирования
        (
            'simpleImputer_after_ord',  # Название шага
            SimpleImputer(missing_values=-1, strategy='most_frequent')  # Заполняем пропуски, которые были закодированы как -1
        )
    ]
)
ord_pipe


In [ ]:
# 4.Объединим кодирование и масштабирование в один пайплайн с подготовкой данных data_preprocessor
data_preprocessor = ColumnTransformer(
    [
        # Шаг 1: Применяем one-hot encoding для выбранных категориальных столбцов
        ('ohe', ohe_pipe, ohe_columns),  # Применяем ohe_pipe к ohe_columns
        # Шаг 2: Применяем порядковое кодирование для выбранных категориальных столбцов
        ('ord', ord_pipe, ord_columns),  # Применяем ord_pipe к ord_columns
        # Шаг 3: Применяем MinMaxScaler для числовых столбцов
        ('num', MinMaxScaler(), num_columns)
    ],
    remainder='passthrough'  # Пропускаем остальные столбцы без изменений
)
data_preprocessor

In [ ]:
# 5. Создаем итоговый пайплан. Базовая модель - DecisionTreeClassifier
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])
pipe_final

In [ ]:
# 6. Создаем переменную `param_grid` для регрессионной задачи
#  — это набор возможных моделей и гиперпараметров для каждой модели
#Проверь актуализируй параметр X_train в 'select_kbest__k'
# Обновляем param_grid для поиска оптимального количества признаков (k)

param_grid = [
    {
        'models': [KNeighborsRegressor()],
        'models__n_neighbors': range(2, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
   },
    {
        'models': [DecisionTreeRegressor(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 10),
        'models__max_features': range(2, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
        
    },
   
    {
        'models': [LGBMRegressor(random_state=RANDOM_STATE, n_jobs=-1, verbose=-1)],
        'models__max_depth': range(1, 50),
        'models__n_estimators': range(10, 100),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough'],
        
    }

   ]

param_grid


In [ ]:
# 1. Создаём списки с названиями признаков
ohe_columns = ['dept', 'last_year_promo', 'last_year_violations']
ord_columns = ['level', 'workload']
num_columns = ['employment_years', 'supervisor_evaluation', 'salary']

In [ ]:
#7. Создаем функцию для расчета новой метрики
""""
SMAPE (англ. symmetric mean absolute percentage error, «симметричное среднее абсолютное процентное отклонение»). 
Метрика SMAPE вычисляется так:
`SMAPE=100𝑛∑𝑖=1𝑛∣𝑦𝑖−𝑦𝑖^∣(∣𝑦𝑖∣+∣𝑦^𝑖∣)/2,SMAPE=n100∑i=1n(∣yi∣+∣y^i∣)/2∣yi−yi^∣`
где:
* 𝑦𝑖yi — фактическое значение целевого признака для объекта с порядковым номером 𝑖i в выборке;
* 𝑦𝑖^yi^ — предсказанное значение целевого признака для объекта с порядковым номером 𝑖i в выборке;
* 𝑛n — количество объектов в выборке;
* ∑𝑖=1𝑛∑i=1n — сумма значений, полученная в результате операций, которые следуют за этим знаком, для всех объектов с порядковым номером от 𝑖i до 𝑛n в выборке.
"""
def smape(y_true, y_pred):
    """
    Функция принимает на вход массивы NumPy или объекты Series в pandas и возвращает значение метрики SMAPE
    (симметричное среднее абсолютное процентное отклонение)

    Args:
        y_true(np.array / pandas.Series): Массив с фактическими значениями данных
        y_pred(np.array / pandas.Series): Массив с прогнозируемыми значениями данных 

    Returns:
        None

    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator != 0)
    return np.mean(diff) * 100


# Создаём пользовательскую метрику
smape_score = make_scorer(smape, greater_is_better=False)
smape_score


#### Подбор гиперпараметров для пайплайна

Используем рекомендованную метрику, которая была расчитана ранее `smape_score`

In [ ]:
grid_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    n_jobs=-1,
    n_iter=100,
    cv=10,
    scoring=smape_score,
    random_state=RANDOM_STATE
)

In [ ]:
grid_search.fit(job_X_train_ml, job_y_train_ml)


In [ ]:
print('Лучшая модель и её параметры:\n\n', grid_search.best_estimator_)
print(line_1)
print('Параметры лучшей модели:', grid_search.best_params_)
print(line_1)
print('Метрика лучшей модели по кросс-валидации на обучающих данных:', grid_search.best_score_*-1)
print(line_1)
job_y_test_pred = grid_search.best_estimator_.predict(job_X_test_ml)
print(f'SMAPE на тестовой выборке: {smape(job_y_test_ml, job_y_test_pred)}')
print(line_1)
print(line_2)

### Анализ важности признаков


In [ ]:
# Получаем лучшую модель и пайплайн предобработки данных из grid_search
# model - сама модель, preprocessor_pipe - пайплайн предобработки данных (например, масштабирование, кодирование признаков)
model = grid_search.best_estimator_.named_steps['models']
preprocessor_pipe = grid_search.best_estimator_.named_steps['preprocessor']

# Масштабируем и обрабатываем тестовые данные с помощью пайплайна
# Это те же шаги предобработки, которые применялись к тренировочным данным
X_test_scaled = preprocessor_pipe.transform(job_X_test_ml)

# Получаем названия признаков после предобработки (если были категории, они могли стать OHE-признаками)
feature_names = preprocessor_pipe.get_feature_names_out()

# Создаем объяснитель для модели с помощью SHAP
explainer = shap.TreeExplainer(model)

# Получаем SHAP-значения для тестовых данных
# SHAP-значения показывают вклад каждого признака в предсказания модели
shap_values = explainer.shap_values(X_test_scaled)

# Визуализируем важность признаков с использованием SHAP-значений
# plot_type="bar" создаёт гистограмму, показывающую суммарную важность каждого признака
# show=False позволяет сначала внести изменения в график перед его отображением
shap.summary_plot(shap_values, X_test_scaled, feature_names=feature_names, plot_type="bar", show=False)

# Добавляем заголовок и подписи осей с помощью matplotlib
# Ось X показывает суммарные SHAP-значения (влияние признаков)
# Ось Y показывает сами признаки, отсортированные по важности
plt.title("Суммарная важность каждого признака")
plt.xlabel("Суммарное вляние признака (SHAP)")  # Подпись оси X
plt.ylabel("Признаки")  # Подпись оси Y

# Отображаем график с добавленными подписями
plt.show()


In [ ]:
grid_search.best_estimator_.named_steps['models']

**Вывод:**
Основной вклад в предсказания модели вносят:
- **`num__supervisor_evaluation`** (оценка от руководителя) — самый влиятельный признак.
- **`num__employment_years`** (стаж работы в компании) и **`num__salary`** (зарплата) — также важны.
- **`ohe__last_year_violations_yes`** (наличие нарушений) и **`ord__level`** (уровень должности) оказывают умеренное влияние.
- Признаки, связанные с отделом (**`ohe__dept_*`**), имеют наименьшее влияние.

Модель опирается в основном на производственные и финансовые показатели сотрудников.

## Шаг 1.6. Выводы

Промежуточные выводы о выбранной модели: 
- Ввели новую метрику — **SMAPE** (симметричное среднее абсолютное процентное отклонение).
- Построили пайплайн;
- Провели подбор гиперпараметров для трех моделей *KNeighborsRegressor*,*DecisionTreeRegressor* и *LGBMRegressor*, 
- Наилучшие результаты показала модель `LGBMRegressor(max_depth=19, n_estimators=77, random_state=42)`, с **SMAPE** на тестовых данных равным `11.298824391526017`.
- Далее мы более тщательно настроили гиперпараметры для `LGBMRegressor`:
- Лучшая модель оказалась с параметрами `LGBMRegressor(max_depth=34, n_estimators=99, n_jobs=-1, random_state=42,verbose=-1)`
- Значение **SMAPE** на тестовой выборке составило **`11.04926727441189`**.


- *LGBM, будучи градиентным бустингом на основе деревьев решений, способен моделировать сложные нелинейные взаимосвязи в данных*. Модель формирует ансамбль деревьев, каждое из которых улучшает результаты предыдущего, что помогает точнее предсказывать сложные зависимости. Поскольку на этапе анализа корреляций были выявлены нелинейные зависимости, LGBM оказался более подходящим для задачи, чем линейные модели. В целом, LGBM автоматически учитывает взаимодействия между признаками, что позволяет лучше адаптироваться к нелинейностям в данных.

# Задача 2: предсказание увольнения сотрудника из компании

## Шаг 2.1. Загрузка данных

### 2.1.1. Загрузка данных

In [ ]:
# Загружаем датасеты для второй задачи
try:
    quit_X_train = pd.read_csv('datasets/train_quit.csv')
    quit_y_test = pd.read_csv('datasets/test_target_quit.csv')

except:
    quit_X_train = pd.read_csv('https://code.s3.yandex.net/datasets/train_quit.csv')
    quit_y_test = pd.read_csv('https://code.s3.yandex.net/datasets/test_target_quit.csv')

# Копируем тестовую выборку    
quit_X_test = job_X_test_ml.copy(deep=True)

### 2.1.2. Общая информация о датасетах

#### `quit_X_train`

In [ ]:
get_info(quit_X_train)

#### `quit_y_test`

In [ ]:
get_info(quit_y_test)

### Выводы по датасетам:

1. **Датасет `quit_X_train` (тренировочная выборка)**:

- **Размер**: 4000 записей и 10 столбцов.
- **Признаки**:
  - **id**: Уникальный идентификатор сотрудника (числовой).
  - **dept**: Отдел, в котором работает сотрудник (категориальный).
  - **level**: Уровень должности (категориальный).
  - **workload**: Уровень загруженности (категориальный).
  - **employment_years**: Количество лет работы в компании (целочисленный).
  - **last_year_promo**: Было ли повышение за последний год (категориальный).
  - **last_year_violations**: Были ли нарушения за последний год (категориальный).
  - **supervisor_evaluation**: Оценка работы сотрудника от руководителя (целочисленный, от 1 до 5).
  - **salary**: Зарплата сотрудника (целочисленный).
  - **quit**: Целевой признак — увольнение (категориальный).

- **Статистическое описание числовых признаков**:
  - **employment_years**: Средний стаж работы составляет 3.7 года. Минимальное значение — 1 год, максимальное — 10 лет.
  - **supervisor_evaluation**: Средняя оценка от руководителя — 3.47. Оценки варьируются от 1 до 5.
  - **salary**: Средняя зарплата — 33,805 единиц, с минимальной зарплатой в 12,000 и максимальной в 96,000 единиц.

- **Пропуски**: Пропущенных данных нет — все столбцы полностью заполнены.
- **Дубликаты**: Явных дубликатов не обнаружено.

2. **Датасет `quit_y_test` (тестовая выборка)**:

- **Размер**: 2000 записей и 2 столбца.
- **Признаки**:
  - **id**: Уникальный идентификатор сотрудника (числовой).
  - **quit**: Целевой признак — увольнение (категориальный).

- **Пропуски**: Пропущенных данных нет — все столбцы полностью заполнены.

**Общий вывод:**
- Данные чистые: отсутствуют пропуски и дубликаты.
- В тренировочной выборке представлены как числовые (стаж работы, зарплата, оценка руководителя), так и категориальные признаки (отдел, уровень должности, загруженность и т.д.).
- Средний стаж сотрудников составляет около 3.7 лет, с зарплатой около 33,805 единиц.
- Признак **quit** является целевым, и его задача — предсказать, уволится сотрудник или нет.

Можно приступить к дальнейшему анализу данных или построению модели для предсказания увольнений сотрудников.

## Шаг 2.2. Предобработка данных

#### `quit_X_train`

In [ ]:
# Выявление неявных дубликатов. Создадим словарь с уникальными значениями для каждого категориального признака
unique_values_dict_1 = {col: quit_X_train[col].unique().tolist() for col in quit_X_train.columns if quit_X_train[col].dtype == 'object'}
unique_values_dict_1

**Проверка и исправление опечаток**: Необходимо исправить опечатку `'sinior'` на `'senior'` в признаке `level`, чтобы избежать появления новых категорий и ошибок при обучении модели.
  


In [ ]:
# Повторное исправление опечатки в колонке level
quit_X_train['level'] = quit_X_train['level'].replace('sinior', 'senior')

# Проверим уникальные значения в колонке 'level' после исправления
quit_X_train['level'].unique()

#### `quit_y_test`

In [ ]:
quit_y_test.info()
quit_y_test.head()


In [ ]:
quit_y_test['quit'].unique()

#### `quit_X_test`

In [ ]:
quit_X_test.reset_index().head()

In [ ]:
quit_X_test = job_X_test_ml.merge(quit_y_test, on='id', how='inner')
quit_X_test.info()

In [ ]:
# Кодируем таргет через LabelEncoder
le = LabelEncoder()
le.fit(["no", "yes"])
quit_X_train['quit'] = le.transform(quit_X_train['quit'])
quit_X_test['quit'] = le.transform(quit_X_test['quit'])

In [ ]:
quit_X_train.info()
quit_X_train.head()


In [ ]:
quit_X_test.info()
quit_X_test.head()

In [ ]:
# Грамотно индексируем выборки
quit_X_train = quit_X_train.set_index('id')
quit_X_test = quit_X_test.set_index('id')

display(quit_X_train.head(1))
display(quit_X_test.head(1))

## Шаг 2.3. Исследовательский анализ данных

#### Количественные данные

In [ ]:
#Гистограммы
plot_histograms_for_df(quit_X_train,'Данные', quit_discrete_columns, quit_continuous_columns)

#Выбросы
plot_interactive_plots(quit_X_train,'Данные', quit_discrete_columns, quit_continuous_columns)


#### Категориальные данные

In [ ]:
plot_categorical_features(quit_X_train,'Данные', quit_categorical_features)

#### Корреляционный анализ

In [ ]:
perform_correlation_analysis(quit_X_train, quit_discrete_columns, quit_continuous_columns)

### Выводы EDA

1. **employment_years** (стаж работы):
   - Большинство сотрудников работают 1-3 года. Стаж свыше 6 лет встречается реже.
   - Стаж положительно коррелирует с **salary** (зарплатой) и **supervisor_evaluation** (оценкой руководителя).

2. **supervisor_evaluation** (оценка руководителя):
   - Основная часть сотрудников имеет высокие оценки (3-4), что свидетельствует о хорошей производительности.
   - Низкие оценки сотрудников могут быть связаны с повышенной вероятностью увольнения.

3. **salary** (зарплата):
   - Зарплаты варьируются от 12,000 до 96,000. Средняя зарплата составляет 33,805, медиана — 30,000.
   - Более высокая зарплата связана с меньшей вероятностью увольнения (**quit**).

4. **dept** (отдел):
   - Большинство сотрудников работают в отделах **sales** (35.9%) и **technology** (23.2%).
   - Отдел **hr** представлен наименьшей долей сотрудников (11.6%).

5. **last_year_violations** и **last_year_promo** (нарушения и повышения за последний год):
   - 86.4% сотрудников не имеют нарушений трудовой дисциплины.
   - Только 2.8% сотрудников были повышены за последний год, что указывает на низкую частоту карьерных изменений.

6. **Корреляция признаков**:
   - **salary** (зарплата) и **employment_years** (стаж работы) имеют положительную корреляцию с **quit** (увольнением).
   - Значимая корреляция также наблюдается между **level** (уровень должности), **workload** (загруженность) и **salary** (зарплата).



In [ ]:
# Описание портрета уволившихся
plot_quit_analysis_in_columns(quit_X_train, target='quit', plot_type='box', cols=2)


**Анализ каждого графика:**

1. **`employment_years` по `dept` с учётом `quit`**:
   - Уволившиеся сотрудники имеют меньший стаж работы (в среднем 1-3 года) во всех отделах, особенно в отделах **sales** и **purchasing**.
   - В отделе **technology** сотрудники остаются работать дольше, но всё равно увольняются с меньшим стажем.

2. **`supervisor_evaluation` по `dept` с учётом `quit`**:
   - В целом, оценки от руководителей практически не отличаются для уволившихся и оставшихся сотрудников во всех отделах, за исключением отдела **hr**, где оценки у уволившихся ниже.
   - Это может указывать на то, что оценка от руководителя не является главным фактором увольнения.

3. **`salary` по `dept` с учётом `quit`**:
   - Уволившиеся сотрудники получают значительно меньшие зарплаты по сравнению с оставшимися, особенно в отделах **technology** и **marketing**.
   - В отделах **sales** и **purchasing** также наблюдается заметное снижение зарплат среди уволившихся.

4. **`employment_years` по `level` с учётом `quit`**:
   - Уволившиеся сотрудники на уровне **junior** имеют минимальный стаж (1-2 года), в то время как сотрудники уровня **middle** и **senior** остаются в компании дольше.
   - Вероятно, сотрудники уровня **junior** увольняются на ранних этапах карьеры из-за недостаточной удовлетворённости работой или недостатка карьерного роста.

5. **`supervisor_evaluation` по `level` с учётом `quit`**:
   - Разница в оценках от руководителя между уволившимися и оставшимися сотрудниками минимальна, что подтверждает предыдущий вывод о незначительном влиянии этого признака на увольнение.
   - На уровне **junior** оценки руководителей остаются одинаковыми для обеих групп.

6. **`salary` по `level` с учётом `quit`**:
   - Уволившиеся сотрудники на уровне **junior** получают на 10 000–15 000 меньше, чем оставшиеся. Разница особенно заметна на уровнях **junior** и **middle**.
   - Это указывает на то, что сотрудники с низким уровнем дохода более склонны к увольнению, особенно на низших уровнях.

7. **`employment_years` по `workload` с учётом `quit`**:
   - Уволившиеся сотрудники имеют меньший стаж на всех уровнях загруженности (**low**, **medium**, **high**), особенно на низком уровне загруженности.
   - Возможно, сотрудники с низкой загруженностью менее мотивированы или менее привязаны к работе.

8. **`salary` по `workload` с учётом `quit`**:
   - Зарплаты уволившихся сотрудников заметно ниже по всем уровням загруженности. Особенно заметна разница среди сотрудников с низкой загруженностью (**low**).
   - Сотрудники с высокой загруженностью и низкой зарплатой также чаще увольняются.

9. **`employment_years` по `last_year_promo` с учётом `quit`**:
   - Большинство уволившихся сотрудников не получали повышения за последний год, что может быть одной из причин увольнения.
   - Стаж уволившихся сотрудников, которые не получали повышения, значительно ниже.

10. **`supervisor_evaluation` по `last_year_promo` с учётом `quit`**:
   - Оценка от руководителя не оказывает значительного влияния на увольнение, как и в предыдущих графиках. Независимо от наличия повышения, оценки остаются стабильными.

11. **`salary` по `last_year_promo` с учётом `quit`**:
   - Зарплата уволившихся сотрудников, не получивших повышения, значительно ниже, чем у оставшихся.
   - Сотрудники, получившие повышение, имеют более высокие зарплаты, но среди них уволившихся немного.

12. **`employment_years` по `last_year_violations` с учётом `quit`**:
   - Сотрудники, совершившие нарушения за последний год, чаще увольняются, особенно с меньшим стажем работы.
   - Стаж работы у оставшихся сотрудников без нарушений выше.

13. **`supervisor_evaluation` по `last_year_violations` с учётом `quit`**:
   - Сотрудники с нарушениями получают более низкие оценки от руководителей, особенно среди уволившихся.

14. **`salary` по `last_year_violations` с учётом `quit`**:
   - Уволившиеся сотрудники с нарушениями за последний год получают заметно более низкие зарплаты, что может быть связано с дисциплинарными мерами или общим уровнем удовлетворённости.

---

**Общий портрет уволившихся сотрудников:**

1. **Зарплата**:
   - Основным фактором, связанным с увольнением, является низкий уровень зарплаты. Во всех отделах, на всех уровнях должностей и загруженности, уволившиеся сотрудники зарабатывают значительно меньше, чем оставшиеся.

2. **Стаж работы**:
   - Сотрудники с небольшим стажем (1–3 года) гораздо чаще увольняются. Это особенно заметно на уровнях **junior** и среди сотрудников с низкой загруженностью.

3. **Уровень должности**:
   - Сотрудники уровня **junior** значительно чаще увольняются, особенно если они получают низкие оценки от руководства и зарплату ниже среднего.

4. **Загруженность**:
   - Сотрудники с низкой загруженностью также чаще увольняются, особенно если их зарплата не соответствует ожиданиям.

5. **Нарушения и повышение**:
   - Сотрудники, которые совершили нарушения трудовой дисциплины за последний год, а также те, кто не получил повышения, чаще увольняются, особенно если они получают более низкие зарплаты и имеют небольшой стаж.

Этот анализ помогает выделить ключевые факторы, влияющие на увольнение, и может быть полезен для улучшения условий труда и удержания сотрудников в компании.

## Шаг 2.4. Добавление нового входного признака

Допустим, job_satisfaction_rate и quit действительно связаны и вы получили необходимое значение метрики в первой задаче. Тогда добавьте job_satisfaction_rate, предсказанный лучшей моделью первой задачи, к входным признакам второй задачи.

##### Тренеровочные данные

In [ ]:
# Проверим совпадение ID работников в обучающих датасетах
common_ids = set(quit_X_train.index).intersection(set(job_X_train.index))
print("Общие 'id':", len(common_ids))

Вывод: не совподают. Спргнозируем job_satisfaction_rate для тренеровочных данных.

In [ ]:
#Спрогнозируем job_satisfaction_rate для quit_X_train
quit_train_J = grid_search.best_estimator_.predict(quit_X_train)
quit_train_J

In [ ]:
#Подготовим данные
quit_X_train_ml = quit_X_train.copy(deep=True)
quit_X_train_ml['job_satisfaction_rate'] = quit_train_J
quit_X_train_ml.head()

#### Тестовые данные

Добавим job_satisfaction_rate значение предсказанного целевого признака первой модели job_y_test_pred

In [ ]:
# Проверим, количество схожих ID работников в тестовых датасетах
common_ids = set(quit_X_test.index).intersection(set(job_X_test_ml.index))
print(len(quit_X_test))
print(len(job_X_test_ml))
print("Общие 'id':", len(common_ids))

Индексы совподают. Загрузим данные в job_satisfaction_rate

In [ ]:
quit_X_test_ml = quit_X_test.copy(deep=True)
quit_X_test_ml['job_satisfaction_rate'] = job_y_test_pred
quit_X_test_ml.head()

In [ ]:
#Проверим размерность
# Оптимизированная проверка размерностей DataFrame'ов
for name, dataframe in zip(['quit_X_train_ml', 'quit_X_test_ml'],
                           [quit_X_train_ml, quit_X_test_ml]):
    print(f"Размер {name}: {dataframe.shape}")

## Шаг 2.5. Подготовка данных

In [ ]:
# Выделяем целевой признак и убираем в тренировочной выборке
quit_y_train_ml = quit_X_train_ml['quit']
quit_X_train_ml = quit_X_train_ml.drop('quit', axis=1)


In [ ]:
# Выделяем целевой признак и убираем в тестовой выборке
quit_y_test_ml = quit_X_test['quit']
quit_X_test_ml = quit_X_test_ml.drop('quit', axis=1)


In [ ]:
#Проверим размерность
# Оптимизированная проверка размерностей DataFrame'ов
for name, dataframe in zip(['quit_X_train_ml', 'quit_y_train_ml', 'quit_X_test_ml','quit_y_test_ml'],
                           [quit_X_train_ml, quit_y_train_ml, quit_X_test_ml,quit_y_test_ml]):
    print(f"Размер {name}: {dataframe.shape}")

In [ ]:

display('Количество дубликатов quit_X_train_ml после корректировок:', quit_X_train_ml.duplicated().sum())

In [ ]:
#Удаляем дубликаты
quit_X_train_ml.drop_duplicates(inplace=True, ignore_index=False)
# Проверка
display('Количество дубликатов quit_X_train_ml после удаления:', quit_X_train_ml.duplicated().sum())


In [ ]:
#Отфильтруем в quit_y_train_ml строки так, чтобы остались только
#  те строки, которые имеют соответствующие признаки в quit_X_train_ml

quit_y_train_ml = quit_y_train_ml[quit_y_train_ml.index.isin(quit_X_train_ml.index)]

In [ ]:
#Проверим размерность
# Оптимизированная проверка размерностей DataFrame'ов
for name, dataframe in zip(['quit_X_train_ml', 'quit_y_train_ml', 'quit_X_test_ml','quit_y_test_ml'],
                           [quit_X_train_ml, quit_y_train_ml, quit_X_test_ml,quit_y_test_ml]):
    print(f"Размер {name}: {dataframe.shape}")

In [ ]:
#Настройка пайпа с учетом того, что добавили новый признак в выборки - job_satisfaction_rate
## 1. В числовые данные  num_columns добавляем job_satisfaction_rate
num_columns.append('job_satisfaction_rate')

## 2. После добавления нового признака важно обновить пайплайн, 
# чтобы этот признак корректно обрабатывался и использовался моделью 
# LogisticRegression для предсказаний.

pipe_final_1 = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', LogisticRegression(random_state=RANDOM_STATE))
])

In [ ]:
num_columns


## Шаг 2.6. Обучение модели

#### Подбор гиперпараметров для пайплайна

In [ ]:
param_grid_1 = [
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 5),
        
    },
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 5),
        'models__max_features': range(2, 5),
        
    },
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE, solver='liblinear', penalty='l1')],
        'models__C': range(1, 5),
    },
    {
        'models': [LGBMClassifier(random_state=RANDOM_STATE, n_jobs=-1, verbose=-1)],
        'models__max_depth': range(1, 21),
        'models__n_estimators': range(80, 120),
        
    }
]


In [ ]:

grid_search_1 = RandomizedSearchCV(
    pipe_final_1,
    param_grid_1,
    n_jobs=-1,
    n_iter=100,
    cv=10,
    scoring='roc_auc',
    random_state=RANDOM_STATE
)


In [ ]:

grid_search_1.fit(quit_X_train_ml, quit_y_train_ml)


In [ ]:
print('Лучшая модель и её параметры:\n\n', grid_search_1.best_estimator_)
print(line_1)
print('Параметры лучшей модели:', grid_search_1.best_params_)
print(line_1)
print('Метрика ROC-AUC лучшей модели по кросс-валидации на обучающих данных:', grid_search_1.best_score_)
print(line_1)


### Анализ важности признаков


In [ ]:
# Получаем лучшую модель и пайплайн предобработки данных из grid_search
# model - сама модель, preprocessor_pipe - пайплайн предобработки данных (например, масштабирование, кодирование признаков)
model = grid_search_1.best_estimator_.named_steps['models']
preprocessor_pipe = grid_search_1.best_estimator_.named_steps['preprocessor']

# Масштабируем и обрабатываем тестовые данные с помощью пайплайна
# Это те же шаги предобработки, которые применялись к тренировочным данным
X_test_scaled = preprocessor_pipe.transform(quit_X_test_ml)

# Получаем названия признаков после предобработки (если были категории, они могли стать OHE-признаками)
feature_names = preprocessor_pipe.get_feature_names_out()

# Создаем объяснитель для модели с помощью SHAP
explainer = shap.TreeExplainer(model)

# Получаем SHAP-значения для тестовых данных
# SHAP-значения показывают вклад каждого признака в предсказания модели
shap_values = explainer.shap_values(X_test_scaled)

# Визуализируем важность признаков с использованием SHAP-значений
# plot_type="bar" создаёт гистограмму, показывающую суммарную важность каждого признака
# show=False позволяет сначала внести изменения в график перед его отображением
shap.summary_plot(shap_values, X_test_scaled, feature_names=feature_names, plot_type="bar", show=False)

# Добавляем заголовок и подписи осей с помощью matplotlib
# Ось X показывает суммарные SHAP-значения (влияние признаков)
# Ось Y показывает сами признаки, отсортированные по важности
plt.title("Суммарная важность каждого признака")
plt.xlabel("Суммарное вляние признака (SHAP)")  # Подпись оси X
plt.ylabel("Признаки")  # Подпись оси Y

# Отображаем график с добавленными подписями
plt.show()


In [ ]:
grid_search_1.best_estimator_.named_steps['models']

## Шаг 2.7. Выводы

**Основные факторы, влияющие на предсказание увольнений:**
- **Стаж работы** (ключевой фактор).
- **Уровень должности** и **удовлетворенность** работой также важны.
- **Зарплата** имеет умеренное влияние.
- **Отдел** и **нарушения** оказывают минимальное влияние на предсказания модели.

# Общий вывод

1. **Задача 1: Предсказание уровня удовлетворенности сотрудника**:
    - Ввели новую метрику — **SMAPE** (симметричное среднее абсолютное процентное отклонение).
    - Построили пайплайн;
    - Провели подбор гиперпараметров для трех моделей *KNeighborsRegressor*,*DecisionTreeRegressor* и *LGBMRegressor*, 
    - Наилучшие результаты показала модель `LGBMRegressor(max_depth=19, n_estimators=77, random_state=42)`, с **SMAPE** на тестовых данных равным `11.298824391526017`.
    - Лучшая модель оказалась с параметрами `LGBMRegressor(max_depth=34, n_estimators=99, n_jobs=-1, random_state=42,verbose=-1)`
    - Значение **SMAPE** на тестовой выборке составило **`11.04926727441189`**.

2. **Задача 2: Предсказание увольнения**:
   - Лучшая модель: **`LGBMRegressor(max_depth=2, n_estimators=96, n_jobs=-1, random_state=42, verbose=-1)`**.
   - **ROC-AUC** на тестовой выборке: **0.93**.
   - Важнейшие признаки: стаж работы и уровень должности.

3. **Рекомендации**:
   - Обращать внимание на оценку труда и удовлетворенность сотрудников.
   - Увеличить удержание сотрудников через соц. программы и премии.
   - Обратить внимание на условия работы в отделе продаж для снижения текучки.